In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])
traindata['attack_cat'] = lb_make.fit_transform(traindata['attack_cat'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])
testdata['attack_cat'] = lb_make.fit_transform(testdata['attack_cat'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-2]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-2]

scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

Y = np.array(Y)
y = np.array(y)
Y = to_categorical(Y)
y = to_categorical(y)

X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))

Using TensorFlow backend.


In [2]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(41, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation="softmax"))
cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
a=[]
folds=5

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu", input_shape=(41, 1))`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
for i in range(folds):
    print("Training on Fold: ",i+1)
    XTrain, XVal, yTrain, yVal = train_test_split(X, Y, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
    cnn.fit(XTrain, yTrain, nb_epoch=200, validation_data=(XVal, yVal))
    scores = cnn.evaluate(XVal, yVal, verbose=0)
    print("%s: %.2f%%" % (cnn.metrics_names[1], scores[1]*100))
    a.append(scores[1] * 100)
    print("======="*12, end="\n\n\n")

Training on Fold:  1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 140272 samples, validate on 35069 samples
Epoch 1/200
140272/140272 [==============================] - 45s 317us/step - loss: 1.3947 - accuracy: 0.5301 - val_loss: 1.3491 - val_accuracy: 0.5348
Epoch 2/200
140272/140272 [==============================] - 42s 299us/step - loss: 1.3508 - accuracy: 0.5338 - val_loss: 1.3245 - val_accuracy: 0.5357
Epoch 3/200
140272/140272 [==============================] - 42s 300us/step - loss: 1.3372 - accuracy: 0.5358 - val_loss: 1.3201 - val_accuracy: 0.5369
Epoch 4/200
140272/140272 [==============================] - 43s 304us/step - loss: 1.3296 - accuracy: 0.5394 - val_loss: 1.3141 - val_accuracy: 0.5428
Epoch 5/200
140272/140272 [==============================] - 43s 304us/step - loss: 1.3246 - accuracy: 0.5429 - val_loss: 1.3110 - val_accuracy: 0.5543
Epoch 6/200
140272/140272 [==============================] - 43s 304us/step - loss: 1.3203 - accuracy: 0.5436 - val_loss: 1.3072 - val_accuracy: 0.5484
Epoch 7/200
140272/140272 [==========

140272/140272 [==============================] - 40s 286us/step - loss: 0.8885 - accuracy: 0.6512 - val_loss: 0.8409 - val_accuracy: 0.6626
Epoch 108/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8867 - accuracy: 0.6518 - val_loss: 0.8452 - val_accuracy: 0.6674
Epoch 109/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8883 - accuracy: 0.6509 - val_loss: 0.8535 - val_accuracy: 0.6588
Epoch 110/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8903 - accuracy: 0.6500 - val_loss: 0.8401 - val_accuracy: 0.6665
Epoch 111/200
140272/140272 [==============================] - 40s 283us/step - loss: 0.8887 - accuracy: 0.6515 - val_loss: 0.8342 - val_accuracy: 0.6666
Epoch 112/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8883 - accuracy: 0.6520 - val_loss: 0.8546 - val_accuracy: 0.6664
Epoch 113/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.88

140272/140272 [==============================] - 40s 283us/step - loss: 0.8795 - accuracy: 0.6536 - val_loss: 0.8361 - val_accuracy: 0.6681
Epoch 13/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8821 - accuracy: 0.6541 - val_loss: 0.8297 - val_accuracy: 0.6687
Epoch 14/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8791 - accuracy: 0.6546 - val_loss: 0.8314 - val_accuracy: 0.6668
Epoch 15/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8808 - accuracy: 0.6545 - val_loss: 0.8462 - val_accuracy: 0.6486
Epoch 16/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.8788 - accuracy: 0.6540 - val_loss: 0.8240 - val_accuracy: 0.6658
Epoch 17/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8783 - accuracy: 0.6553 - val_loss: 0.8444 - val_accuracy: 0.6507
Epoch 18/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8801 - a

...

140272/140272 [==============================] - 40s 284us/step - loss: 0.8912 - accuracy: 0.6409 - val_loss: 0.8456 - val_accuracy: 0.6437
Epoch 130/200
140272/140272 [==============================] - 40s 282us/step - loss: 0.8915 - accuracy: 0.6409 - val_loss: 0.8530 - val_accuracy: 0.6569
Epoch 131/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8916 - accuracy: 0.6416 - val_loss: 0.8441 - val_accuracy: 0.6539
Epoch 132/200
140272/140272 [==============================] - 41s 289us/step - loss: 0.8927 - accuracy: 0.6397 - val_loss: 0.8364 - val_accuracy: 0.6526
Epoch 133/200
140272/140272 [==============================] - 40s 283us/step - loss: 0.8908 - accuracy: 0.6404 - val_loss: 0.8385 - val_accuracy: 0.6545
Epoch 134/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8918 - accuracy: 0.6404 - val_loss: 0.8402 - val_accuracy: 0.6563
Epoch 135/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.88

140272/140272 [==============================] - 40s 284us/step - loss: 0.8920 - accuracy: 0.6417 - val_loss: 0.8342 - val_accuracy: 0.6547
Epoch 35/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8933 - accuracy: 0.6429 - val_loss: 0.8368 - val_accuracy: 0.6523
Epoch 36/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8914 - accuracy: 0.6429 - val_loss: 0.8335 - val_accuracy: 0.6604
Epoch 37/200
140272/140272 [==============================] - 40s 282us/step - loss: 0.8950 - accuracy: 0.6409 - val_loss: 0.8429 - val_accuracy: 0.6499
Epoch 38/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8953 - accuracy: 0.6405 - val_loss: 0.8342 - val_accuracy: 0.6529
Epoch 39/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8958 - accuracy: 0.6411 - val_loss: 0.8349 - val_accuracy: 0.6501
Epoch 40/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8905 - a

140272/140272 [==============================] - 40s 285us/step - loss: 0.8879 - accuracy: 0.6445 - val_loss: 0.8344 - val_accuracy: 0.6585
Epoch 141/200
140272/140272 [==============================] - 41s 289us/step - loss: 0.8908 - accuracy: 0.6419 - val_loss: 0.8308 - val_accuracy: 0.6516
Epoch 142/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8876 - accuracy: 0.6435 - val_loss: 0.8324 - val_accuracy: 0.6497
Epoch 143/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8859 - accuracy: 0.6451 - val_loss: 0.8364 - val_accuracy: 0.6513
Epoch 144/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8893 - accuracy: 0.6438 - val_loss: 0.8563 - val_accuracy: 0.6529
Epoch 145/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8873 - accuracy: 0.6444 - val_loss: 0.8295 - val_accuracy: 0.6496
Epoch 146/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.89

140272/140272 [==============================] - 40s 286us/step - loss: 0.8977 - accuracy: 0.6422 - val_loss: 0.8335 - val_accuracy: 0.6607
Epoch 46/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.8929 - accuracy: 0.6430 - val_loss: 0.8574 - val_accuracy: 0.6391
Epoch 47/200
140272/140272 [==============================] - 40s 285us/step - loss: 0.8969 - accuracy: 0.6420 - val_loss: 0.8311 - val_accuracy: 0.6632
Epoch 48/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8966 - accuracy: 0.6430 - val_loss: 0.8453 - val_accuracy: 0.6577
Epoch 49/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.8955 - accuracy: 0.6427 - val_loss: 0.8280 - val_accuracy: 0.6612
Epoch 50/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.8980 - accuracy: 0.6427 - val_loss: 0.8345 - val_accuracy: 0.6565
Epoch 51/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.8945 - a

140272/140272 [==============================] - 40s 284us/step - loss: 0.8954 - accuracy: 0.6432 - val_loss: 0.8556 - val_accuracy: 0.6483
Epoch 152/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8952 - accuracy: 0.6435 - val_loss: 0.8322 - val_accuracy: 0.6633
Epoch 153/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8925 - accuracy: 0.6432 - val_loss: 0.8314 - val_accuracy: 0.6610
Epoch 154/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.9010 - accuracy: 0.6428 - val_loss: 0.8291 - val_accuracy: 0.6661
Epoch 155/200
140272/140272 [==============================] - 40s 283us/step - loss: 0.8934 - accuracy: 0.6429 - val_loss: 0.8316 - val_accuracy: 0.6610
Epoch 156/200
140272/140272 [==============================] - 40s 284us/step - loss: 0.8925 - accuracy: 0.6433 - val_loss: 0.8332 - val_accuracy: 0.6625
Epoch 157/200
140272/140272 [==============================] - 40s 286us/step - loss: 0.89

In [4]:
print (a)

[64.25047516822815, 66.45470261573792, 66.01842045783997, 65.61635732650757, 66.04693531990051]
